In [56]:
import pandas as pd
import numpy as np

# 데이터 호출

In [20]:
df = pd.read_excel('supply_chain_sample.xlsx')

In [21]:
df['Periods'] = pd.to_datetime(df['target_start_date'].astype(str).apply(lambda x: x[:10]))

In [22]:
df.columns.unique()

Index(['source_factset_entity_id', 'source_entity_name', 'relationship_type',
       'source_fsym_id', 'source_ticker_region', 'source_isin', 'source_cusip',
       'source_sedol', 'target_start_date', 'target_end_date',
       'target_factset_entity_id', 'target_entity_name', 'target_revenue_pct',
       'target_primary_ticker', 'target_country', 'target_entity_type',
       'target_fsym_id', 'target_ticker_region', 'target_isin', 'target_cusip',
       'target_sedol', 'Periods'],
      dtype='object')

In [23]:
df = df[['Periods', 'source_entity_name', 'relationship_type', 'source_ticker_region', 'target_entity_name', 'target_ticker_region']]

In [30]:
df = df.dropna(subset = ['target_ticker_region'])

In [34]:
df = df[df['relationship_type'].isin(['Supplier', 'Customer'])]

In [41]:
df = df[df['Periods'] > '2014-12-31']

In [42]:
df

,Periods,source_entity_name,relationship_type,source_ticker_region,target_entity_name,target_ticker_region
226,2015-03-05,BP Plc,Customer,BP-GB,"Tokyo Electric Power Co. Holdings, Inc.",9501-JP
283,2015-03-05,BP Plc,Supplier,BP-GB,"DAEWOO SHIPBUILDING & MARINE ENGINEERING Co., ...",042660-KR
19,2015-11-03,"Apple, Inc.",Supplier,AAPL-US,TDK Corp.,6762-JP
18,2015-11-03,"Apple, Inc.",Supplier,AAPL-US,"Taiwan Semiconductor Manufacturing Co., Ltd.",2330-TW
184,2015-11-03,"Apple, Inc.",Supplier,AAPL-US,Knowles Corp.,KN-US
...,...,...,...,...,...,...
350,2019-02-12,"Alphabet, Inc.",Customer,GOOGL-US,Moncler SpA,MONC-IT
345,2019-02-12,"Alphabet, Inc.",Customer,GOOGL-US,"Adobe, Inc.",ADBE-US
527,2019-02-12,"Alphabet, Inc.",Customer,GOOGL-US,Air France-KLM SA,AF-FR
458,2019-02-19,"Alphabet, Inc.",Customer,GOOGL-US,Wilderness Holdings Ltd.,WIL-BW


In [43]:
df['relationship_type'].value_counts()

Customer    102
Supplier     76
Name: relationship_type, dtype: int64

In [44]:
df2 = pd.read_csv('SC links company data.txt', sep='\t')

In [45]:
df2['Periods'] = pd.to_datetime(df2['Periods'])

In [46]:
df2.columns.unique()

Index(['Unnamed: 0', 'Company Name', 'Periods', 'Ticker', 'Universe Returns',
       'Country', 'LT Debt to Capital', 'LT Debt to Equity', 'Cash Ratio',
       'Debt Service Ratio', 'Beneish M-Score', 'Net Income Stability',
       'ESG Rating', 'Credit Rating', 'Credit Rating - Numeric'],
      dtype='object')

In [47]:
df2 = df2[['Periods', 'Company Name', 'Ticker', 'Country', 'LT Debt to Capital', 'LT Debt to Equity', 'Cash Ratio', 'Debt Service Ratio', 'Beneish M-Score', 'Net Income Stability', 'ESG Rating', 'Credit Rating']]
df2

,Periods,Company Name,Ticker,Country,LT Debt to Capital,LT Debt to Equity,Cash Ratio,Debt Service Ratio,Beneish M-Score,Net Income Stability,ESG Rating,Credit Rating
0,2014-12-31,"AbbVie, Inc.",ABBV-US,US,0.72,2.77,1.62,5.81,-2.71,5.68,BB,A
1,2015-01-30,"AbbVie, Inc.",ABBV-US,US,0.72,2.77,1.62,5.81,-2.71,5.68,BB,A
2,2015-02-27,"AbbVie, Inc.",ABBV-US,US,0.63,6.06,0.73,1.24,-2.86,4.15,BB,A
3,2015-03-31,"AbbVie, Inc.",ABBV-US,US,0.72,2.77,1.62,5.81,-2.71,5.68,BB,A
4,2015-04-30,"AbbVie, Inc.",ABBV-US,US,0.74,3.11,1.56,5.81,-2.84,5.64,BB,A
...,...,...,...,...,...,...,...,...,...,...,...,...
28130,2021-08-31,Flex Ltd.,FLEX-US,SG,0.51,1.16,0.33,1.64,-2.41,1.35,AA,BBB-
28131,2021-09-30,Flex Ltd.,FLEX-US,SG,0.51,1.16,0.33,1.64,-2.41,1.35,AA,BBB-
28132,2021-10-29,Flex Ltd.,FLEX-US,SG,0.51,1.16,0.33,1.64,-2.41,1.35,AA,BBB-
28133,2021-11-30,Flex Ltd.,FLEX-US,SG,0.52,1.15,0.28,1.64,-2.31,1.14,AA,BBB-


# 합병

In [48]:
# 합병 시 Direction 사용하려면 우선 정렬되어있어야 함.

df = df.sort_values(['Periods'])
df2 = df2.sort_values(['Periods'])

In [49]:
merged_df = pd.merge_asof(df, df2, left_on = ['Periods'], right_on = ['Periods'], direction = 'backward', left_by = 'source_ticker_region', right_by = 'Ticker')

In [50]:
merged_df

,Periods,source_entity_name,relationship_type,source_ticker_region,target_entity_name,target_ticker_region,Company Name,Ticker,Country,LT Debt to Capital,LT Debt to Equity,Cash Ratio,Debt Service Ratio,Beneish M-Score,Net Income Stability,ESG Rating,Credit Rating
0,2015-03-05,BP Plc,Customer,BP-GB,"Tokyo Electric Power Co. Holdings, Inc.",9501-JP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-03-05,BP Plc,Supplier,BP-GB,"DAEWOO SHIPBUILDING & MARINE ENGINEERING Co., ...",042660-KR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-11-03,"Apple, Inc.",Supplier,AAPL-US,TDK Corp.,6762-JP,Apple Inc.,AAPL-US,US,0.23,0.31,0.56,355.32,-2.86,3.51,A,AA+
3,2015-11-03,"Apple, Inc.",Supplier,AAPL-US,"Taiwan Semiconductor Manufacturing Co., Ltd.",2330-TW,Apple Inc.,AAPL-US,US,0.23,0.31,0.56,355.32,-2.86,3.51,A,AA+
4,2015-11-03,"Apple, Inc.",Supplier,AAPL-US,Knowles Corp.,KN-US,Apple Inc.,AAPL-US,US,0.23,0.31,0.56,355.32,-2.86,3.51,A,AA+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,2019-02-12,"Alphabet, Inc.",Customer,GOOGL-US,"Wyndham Destinations, Inc.",WYND-US,Alphabet Inc. Class A,GOOGL-US,US,0.02,0.02,3.42,186.91,-2.89,5.17,A,AA+
174,2019-02-12,"Alphabet, Inc.",Customer,GOOGL-US,L'Oréal SA,OR-FR,Alphabet Inc. Class A,GOOGL-US,US,0.02,0.02,3.42,186.91,-2.89,5.17,A,AA+
175,2019-02-12,"Alphabet, Inc.",Customer,GOOGL-US,Ctrip.com International Ltd.,CTRP-US,Alphabet Inc. Class A,GOOGL-US,US,0.02,0.02,3.42,186.91,-2.89,5.17,A,AA+
176,2019-02-19,"Alphabet, Inc.",Customer,GOOGL-US,Wilderness Holdings Ltd.,WIL-BW,Alphabet Inc. Class A,GOOGL-US,US,0.02,0.02,3.42,186.91,-2.89,5.17,A,AA+


# 피벗

In [65]:
pivot_table = pd.pivot_table(merged_df[['source_entity_name', 'Periods', 'Country', 'LT Debt to Capital', 'LT Debt to Equity', 'Cash Ratio', 'Debt Service Ratio', 'Beneish M-Score', 'Net Income Stability']], index=['source_entity_name', 'Periods'], columns='Country', values=['LT Debt to Capital', 'LT Debt to Equity', 'Cash Ratio', 'Debt Service Ratio', 'Beneish M-Score', 'Net Income Stability'], aggfunc=np.mean)
pivot_table

Beneish M-Score Cash Ratio Debt Service Ratio  \
Country                                    US         US                 US   
source_entity_name Periods                                                    
Alphabet, Inc.     2019-02-11           -2.89       3.42             186.91   
                   2019-02-12           -2.89       3.42             186.91   
                   2019-02-19           -2.89       3.42             186.91   
                   2019-04-01           -2.89       3.40             186.91   
Apple, Inc.        2015-11-03           -2.86       0.56             355.32   
                   2015-11-04           -2.86       0.56             355.32   
                   2016-02-09           -2.89       0.53             355.32   
                   2016-03-31           -2.89       0.53             355.32   
                   2016-04-01           -2.89       0.53             355.32   
                   2016-11-03           -2.89       0.81               7.87   
                   2017-11-07           -2.79       0.91               5.68   
                   2017-11-16           -2.79       0.91               5.68   
                   2017-12-12           -2.73       0.74               2.89   
                   2017-12-13           -2.73       0.74               2.89   
                   2018-11-22           -2.60       0.98               4.48   
                   2019-02-08           -2.58       0.80               4.48   

                              LT Debt to Capital LT Debt to Equity  \
Country                                       US                US   
source_entity_name Periods                                           
Alphabet, Inc.     2019-02-11               0.02              0.02   
                   2019-02-12               0.02              0.02   
                   2019-02-19               0.02              0.02   
                   2019-04-01               0.02              0.02   
Apple, Inc.        2015-11-03               0.23              0.31   
                   2015-11-04               0.23              0.31   
                   2016-02-09               0.26              0.38   
                   2016-03-31               0.26              0.38   
                   2016-04-01               0.26              0.38   
                   2016-11-03               0.33              0.53   
                   2017-11-07               0.36              0.63   
                   2017-11-16               0.36              0.63   
                   2017-12-12               0.39              0.73   
                   2017-12-13               0.39              0.73   
                   2018-11-22               0.41              0.80   
                   2019-02-08               0.42              0.84   

                              Net Income Stability  
Country                                         US  
source_entity_name Periods                          
Alphabet, Inc.     2019-02-11                 5.17  
                   2019-02-12                 5.17  
                   2019-02-19                 5.17  
                   2019-04-01                 5.17  
Apple, Inc.        2015-11-03                 3.51  
                   2015-11-04                 3.51  
                   2016-02-09                 3.78  
                   2016-03-31                 3.78  
                   2016-04-01                 3.78  
                   2016-11-03                 4.93  
                   2017-11-07                 7.81  
                   2017-11-16                 7.81  
                   2017-12-12                 8.22  
                   2017-12-13                 8.22  
                   2018-11-22                 7.96  
                   2019-02-08                 7.94